In [ ]:
import numpy as np
from pygmid import Lookup
from scipy.optimize import minimize
import scipy.constants as sc

# --- 1. Load Process Technology Data ---
# Replace with your actual .mat file names
try:
    nch = Lookup('Chipathon2025/lookup_tables/nfet_03v3.mat')
    pch = Lookup('Chipathon2025/lookup_tables/pfet_03v3.mat')
except FileNotFoundError:
    print("FATAL ERROR: Make sure your nmos_data.mat and pmos_data.mat files are in the correct directory.")
    exit()

# --- 2. Define Target Mixer Specifications ---
Vdd = 3.3                   # Supply Voltage (V)
R_L = 1000                 # Load Resistance (Ohms) - Can also be an optimization variable
Target_CG_dB = 10           # Target Conversion Gain (dB)
Target_IIP3_dBm = -5        # Target Input Third-Order Intercept Point (dBm)
Target_NF_dB = 15           # Target Noise Figure (dB)
Power_Constraint_mW = 5.0   # Maximum Power Consumption (mW)

# Convert targets to linear scale for calculations
Target_CG_linear = 10**(Target_CG_dB / 20)
Target_IIP3_W = 10**(Target_IIP3_dBm / 10) / 1000
I_total_max = (Power_Constraint_mW / 1000) / Vdd

In [ ]:
# --- 3. The Objective Function ---
# This function calculates the "cost" of a given design. The optimizer's goal is to minimize this cost.
def calculate_cost(x, weights):
    """
    Calculates the cost of a design based on its deviation from targets.
    x: array of design variables -> [gm_id_rf, gm_id_lo, L_rf, L_lo]
    weights: dictionary to prioritize different specs -> {'w_cg', 'w_iip3', 'w_nf'}
    """
    gm_id_rf, gm_id_lo, L_rf, L_lo = x
    
    # --- Performance Modeling using Analytical Formulas & pygmid ---
    
    # a) Conversion Gain (CG)
    # CG_linear = (2/pi) * gm_rf * R_L
    # First, calculate the required tail current based on the RF stage's gm/ID
    # To meet the CG, we need a certain gm_rf.
    gm_rf_req = (Target_CG_linear * np.pi) / (2 * R_L)
    id_w_rf = nch.lookup('ID_W', gm_id=gm_id_rf, L=L_rf)
    
    # The total tail current is determined by the required gm and the chosen gm/ID
    I_tail = 2 * gm_rf_req / gm_id_rf
    
    # b) Power Consumption
    power = I_tail * Vdd
    
    # c) Noise Figure (NF) - Simplified model
    # Considers noise from RF stage, LO switches, and load resistors.
    # Gamma (γ) is the excess noise factor.
    gamma_rf = nch.lookup('STH', VGS=1)/4/sc.Boltzmann/300/nch.lookup('GM', VGS=1, L=L_rf)
    # nch.lookup('GAMMA', gm_id=gm_id_rf, L=L_rf)
    gamma_lo = nch.lookup('STH', VGS=1)/4/sc.Boltzmann/300/nch.lookup('GM', VGS=1, L=L_lo)
    # nch.lookup('GAMMA', gm_id=gm_id_lo, L=L_lo)
    gm_rf = gm_rf_req
    
    # Noise contribution from various sources referred to the output
    vn_out_sq_load = 4 * 1.38e-23 * 300 * R_L
    vn_out_sq_rf = 2 * (4 * 1.38e-23 * 300 * gamma_rf * gm_rf) * (1/gm_rf * (2/np.pi) * R_L)**2 # Factor of 2 for differential pair
    vn_out_sq_lo = 4 * (4 * 1.38e-23 * 300 * gamma_lo * nch.lookup('GM', VGS=1, L=L_lo)) * R_L**2 * (1/np.pi)
                        # nch.lookup('GM', gm_id=gm_id_lo, L=L_lo)) * R_L**2 * (1/np.pi)
    
    # Total input-referred noise voltage squared
    vn_in_sq_total = (vn_out_sq_load + vn_out_sq_rf + vn_out_sq_lo) / Target_CG_linear**2
    NF = 10 * np.log10(1 + vn_in_sq_total / (4 * 1.38e-23 * 300 * 50)) # Assuming 50 Ohm source

    # d) Linearity (IIP3) - Highly simplified model
    # IIP3 is complex. A common approximation relates it to the overdrive voltage (Vov = Vgs - Vth).
    # Higher Vov generally improves linearity. Vov is related to 2/(gm/ID).
    Vov_rf = 2 / gm_id_rf
    # This is a rough proxy. We assume higher Vov is better.
    # A more rigorous model would require gm derivatives (gm3).
    # For optimization, we can say IIP3 is proportional to Vov^2.
    # We'll calculate a proxy for IIP3 and compare its deviation from the target proxy.
    IIP3_proxy = Vov_rf**2 
    Target_IIP3_proxy = (2 / 10)**2 # Proxy based on a moderate target gm/ID of 10

    # --- Calculate Cost ---
    # Cost is the weighted sum of squared normalized errors
    err_cg = (gm_rf - gm_rf_req)**2 / gm_rf_req**2 # CG is fixed by gm_req, so we penalize deviation in gm
    err_nf = (NF - Target_NF_dB)**2 / Target_NF_dB**2 if NF > Target_NF_dB else 0 # Only penalize if NF is worse than target
    err_iip3 = (IIP3_proxy - Target_IIP3_proxy)**2 / Target_IIP3_proxy**2

    # Heavily penalize designs that violate the power constraint
    power_penalty = max(0, I_tail - I_total_max)**2 * 1e6

    total_cost = (weights['w_cg'] * err_cg + 
                  weights['w_nf'] * err_nf + 
                  weights['w_iip3'] * err_iip3 +
                  power_penalty)
                  
    return total_cost


In [ ]:
# --- 4. Optimization Setup ---

# Initial guesses for design variables [gm_id_rf, gm_id_lo, L_rf, L_lo]
initial_guess = [12, 10, 0.28, 0.28]

# Bounds for the design variables to keep them realistic
bounds = [(5, 25),      # gm/ID for RF stage (S/A)
          (5, 20),      # gm/ID for LO stage (S/A)
          (0.28, 1), # Length for RF transistors (um)
          (0.28, 1)] # Length for LO transistors (um)

# Weights to prioritize specifications. Adjust these to guide the optimizer.
# Higher weight means more important.
opti_weights = {'w_cg': 1.0, 'w_nf': 1.0, 'w_iip3': 0.5}

# Constraint: Power consumption must be less than or equal to the maximum allowed.
cons = ({'type': 'ineq', 'fun': lambda x: I_total_max - 2 * ( (Target_CG_linear * np.pi) / (2 * R_L) ) / x[0]})

# --- 5. Run the Optimization ---
print("Starting automated optimization...\n")
result = minimize(calculate_cost, 
                  initial_guess, 
                  args=(opti_weights,),
                  method='SLSQP', 
                  bounds=bounds, 
                  constraints=cons,
                  options={'disp': True})

if not result.success:
    print("\nWARNING: Optimization did not converge successfully.")
    print(f"Message: {result.message}")

# --- 6. Display Optimized Results ---
print("\n--- Optimization Complete ---")
optimized_vars = result.x
gm_id_rf_opt, gm_id_lo_opt, L_rf_opt, L_lo_opt = optimized_vars

# Calculate final device sizes and bias based on optimized variables
gm_rf_opt = (Target_CG_linear * np.pi) / (2 * R_L)
I_tail_opt = 2 * gm_rf_opt / gm_id_rf_opt

# RF Stage (M1, M2)
id_w_rf_opt = nch.lookup('ID_W', gm_id=gm_id_rf_opt, L=L_rf_opt)
W_rf_opt = (I_tail_opt / 2) / id_w_rf_opt
# vgs_rf_opt = nch.lookup('VGS', gm_id=gm_id_rf_opt, L=L_rf_opt)

# LO Stage (M3-M6)
id_w_lo_opt = nch.lookup('ID_W', gm_id=gm_id_lo_opt, L=L_lo_opt)
W_lo_opt = (I_tail_opt / 2) / id_w_lo_opt

# Tail Current Source (M_tail)
# Let's choose a gm/ID and longer length for good output resistance
gm_id_tail_opt = 10
L_tail_opt = 2 * L_rf_opt # Example choice
id_w_tail_opt = nch.lookup('ID_W', gm_id=gm_id_tail_opt, L=L_tail_opt)
W_tail_opt = I_tail_opt / id_w_tail_opt


print(f"\nOptimized for {Power_Constraint_mW}mW Power Budget and {Target_CG_dB}dB Gain\n")
print("--- RF Transconductance Stage (M1, M2) ---")
print(f"Optimal gm/ID: {gm_id_rf_opt:.2f} S/A")
print(f"Optimal Length: {L_rf_opt*1:.3f} um")
print(f"Resulting Width: {W_rf_opt*1:.2f} um")
print(f"Bias Current (per device): {I_tail_opt/2*1e3:.3f} mA")
# print(f"Required Vgs: {vgs_rf_opt:.3f} V (approx.)")

print("\n--- LO Switching Stage (M3-M6) ---")
print(f"Optimal gm/ID: {gm_id_lo_opt:.2f} S/A")
print(f"Optimal Length: {L_lo_opt*1:.3f} um")
print(f"Resulting Width: {W_lo_opt*1:.2f} um")

print("\n--- Tail Current Source ---")
print(f"Chosen gm/ID: {gm_id_tail_opt} S/A")
print(f"Chosen Length: {L_tail_opt*1:.3f} um")
print(f"Resulting Width: {W_tail_opt*1:.2f} um")
print(f"Total Tail Current: {I_tail_opt*1e3:.3f} mA")

# print("\n--- Final Performance Estimates ---")
# final_power = I_tail_opt * Vdd
# final_cg = 20 * np.log10((2/np.pi) * gm_rf_opt * R_L)
# print(f"Estimated Power: {final_power*1e3:.2f} mW")
# print(f"Estimated Conversion Gain: {final_cg:.2f} dB")
# # Re-calculate NF for the final design
# gamma_rf = nch.lookup('GAMMA', gm_id=gm_id_rf_opt, L=L_rf_opt)
# gamma_lo = nch.lookup('GAMMA', gm_id=gm_id_lo_opt, L=L_lo_opt)
# vn_out_sq_load = 4 * 1.38e-23 * 300 * R_L
# vn_out_sq_rf = 2 * (4 * 1.38e-23 * 300 * gamma_rf * gm_rf_opt) * (1/gm_rf_opt * (2/np.pi) * R_L)**2
# vn_out_sq_lo = 4 * (4 * 1.38e-23 * 300 * gamma_lo * nch.lookup('GM', gm_id=gm_id_lo_opt, L=L_lo_opt)) * R_L**2 * (1/np.pi)
# vn_in_sq_total = (vn_out_sq_load + vn_out_sq_rf + vn_out_sq_lo) / final_cg**2
# final_nf = 10 * np.log10(1 + vn_in_sq_total / (4 * 1.38e-23 * 300 * 50))
# print(f"Estimated Noise Figure: {final_nf:.2f} dB")


print("\nIMPORTANT: These results are a highly optimized starting point based on analytical models.")
print("Verification and further refinement using a SPICE circuit simulator are essential.")